<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_merlin_01-building-recommender-systems-with-merlin/nvidia_logo.png" style="width: 90px; float: right;"> 

# Building Online Multi-Stage Recsys Components
The figure below represents a **four-stage recommender system**. This is more complex process than only training a single model and deploying it, and it is much more realistic and closer to what's happening in the real-world recommender production systems. The models and data to perform the bottom row of tasks were previously completed in the first notebook [here](...).

![img](./img/OnlineMultiStageRecsys.png)

In this notebook, we are going to prepare the assets on the bottom row to deploy a four-stage recommender system on [Triton Inference Server](https://github.com/triton-inference-server/server). 

To learn more about the four-stage recommender systems, you can listen to Even Oldridge's [Moving Beyond Recommender Models talk](https://www.youtube.com/watch?v=5qjiY-kLwFY&list=PL65MqKWg6XcrdN4TJV0K1PdLhF_Uq-b43&index=7) at KDD'21 and read more [in this blog post](https://eugeneyan.com/writing/system-design-for-discovery/).

In addition to NVIDIA Merlin libraries and the Triton Inference Server client library, we use two external libraries in these series of examples:

- [Feast](https://docs.feast.dev/): an end-to-end open source feature store library for machine learning
- [Redis](https://github.com/redis/redis-py): a low-latency key-value store and ANN index

## Steps
1) [**Feature Store Setup**](#Feature-Store-Setup)
2) [**Redis ANN Index Setup**](#Redis-ANN-Index-Setup)

### Import required libraries and functions

*These notebooks are developed and tested using `merlin-tensorflow:22.11` container on [NVIDIA's docker registry](https://catalog.ngc.nvidia.com/containers?filters=&orderBy=dateModifiedDESC&query=merlin).*

In [2]:
import os
import merlin.models.tf as mm
import nvtabular as nvt
import numpy as np
import tensorflow as tf
import warnings

warnings.filterwarnings("ignore")

from merlin.datasets.ecommerce import transform_aliccp
from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
from nvtabular.ops import *

# for running this example on CPU, comment out the line below
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

2023-01-18 18:44:41.249559: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-18 18:44:44.874696: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 18

In [3]:
# disable INFO and DEBUG logging everywhere
import logging

logging.disable(logging.WARNING)

First, we define our input path and feature repo path.

In [3]:
# Define output path for data
BASE_DIR = os.environ['PWD']
DATA_DIR = os.path.join(BASE_DIR, "data")

Next, we need to load the previously trained assets. If you have your own great, make sure they end up in the same folder structure as the ones we will pull from the publically hosted S3 bucket below

In [ ]:
# Pull data/models from S3
# TODO

## Feature Store Setup

We need to create a Feast feature repository. [Feast](https://feast.dev/) is an end-to-end open source feature store for machine learning. Feast (Feature Store) is a customizable operational data system that re-uses existing infrastructure to manage and serve machine learning features to real-time models.

Our feature repo will live at the defined path below:

In [4]:
# Feature repo path
feature_repo_path = os.path.join(BASE_DIR, "feature_repo")

### Prepare User and Item features

In [5]:
from merlin.models.utils.dataset import unique_rows_by_features

# Load pre-generated User features file
user_features = Dataset(os.path.join(DATA_DIR, "user_features.parquet")).to_ddf().compute()
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories,user_id_raw
0,1,1,1,1,1,1,1,1,1,1,1,1,7
1,2,2,1,1,1,1,1,1,1,2,2,2,6
2,3,3,1,1,1,1,1,1,1,3,3,3,8
3,4,4,1,1,1,1,1,1,1,4,4,4,9
4,5,5,1,1,1,1,1,1,1,5,5,5,5


We will artificially add `datetime` and `created` timestamp columns to our user_features dataframe. This required by Feast to track the user-item features and their creation time and to determine which version to use when we query Feast.

In [6]:
from datetime import datetime

user_features["datetime"] = datetime.now()
user_features["datetime"] = user_features["datetime"].astype("datetime64[ns]")
user_features["created"] = datetime.now()
user_features["created"] = user_features["created"].astype("datetime64[ns]")
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories,user_id_raw,datetime,created
0,1,1,1,1,1,1,1,1,1,1,1,1,7,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
1,2,2,1,1,1,1,1,1,1,2,2,2,6,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
2,3,3,1,1,1,1,1,1,1,3,3,3,8,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
3,4,4,1,1,1,1,1,1,1,4,4,4,9,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
4,5,5,1,1,1,1,1,1,1,5,5,5,5,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434


In [7]:
# Write parquet file to feature_repo
user_features.to_parquet(
    os.path.join(feature_repo_path, "data", "user_features.parquet")
)

In [8]:
# Load pre-generated Item features file
item_features = Dataset(os.path.join(DATA_DIR, "item_features.parquet")).to_ddf().compute()
item_features.head()

,item_id,item_category,item_shop,item_brand,item_id_raw
0,1,1,1,1,7
1,2,2,2,2,8
2,3,3,3,3,6
3,4,4,4,4,5
4,5,5,5,5,9


In [9]:
# Append timestamps
item_features["datetime"] = datetime.now()
item_features["datetime"] = item_features["datetime"].astype("datetime64[ns]")
item_features["created"] = datetime.now()
item_features["created"] = item_features["created"].astype("datetime64[ns]")
item_features.head()

,item_id,item_category,item_shop,item_brand,item_id_raw,datetime,created
0,1,1,1,1,7,2023-01-17 19:46:50.023935,2023-01-17 19:46:50.025315
1,2,2,2,2,8,2023-01-17 19:46:50.023935,2023-01-17 19:46:50.025315
2,3,3,3,3,6,2023-01-17 19:46:50.023935,2023-01-17 19:46:50.025315
3,4,4,4,4,5,2023-01-17 19:46:50.023935,2023-01-17 19:46:50.025315
4,5,5,5,5,9,2023-01-17 19:46:50.023935,2023-01-17 19:46:50.025315


In [10]:
# Write parquet file to feature_repo
item_features.to_parquet(
    os.path.join(feature_repo_path, "data", "item_features.parquet")
)

### Register features
The Feast feature registry is a central catalog of all the feature definitions and their related metadata (read more [here](https://docs.feast.dev/getting-started/architecture-and-components/registry)). We have defined our user and item features definitions in the `user_features.py` and  `item_features.py` files.

With `FeatureView()` users can register data sources in their organizations into Feast, and then use those data sources for both training and online inference. In the `user_features.py` and `item_features.py` files, we are telling Feast where to find user and item features.

Before we move on to the next steps, we need to perform `feast apply`command as directed below.  With that, we register our features, we can apply the changes to create our feature registry and store all entity and feature view definitions in a local SQLite online store called `online_store.db`.

In [11]:
%cd $feature_repo_path
!feast apply

/workdir/feature_repo
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Created entity user_id_raw
Created entity item_id
Created feature view item_features
Created feature view user_features

Deploying infrastructure for item_features
Deploying infrastructure for user_features


### Materialize (Load) features from Parquet into Redis

After we execute `apply` and registered our features and created our online local store, now we need to perform [materialization](https://docs.feast.dev/how-to-guides/running-feast-in-production) operation. This is done to keep our online store up to date and get it ready for prediction. For that we need to run a job that loads feature data from our feature view sources into our online store. As we add new features to our offline stores, we can continuously materialize them to keep our online store up to date by finding the latest feature values for each user. 

When you run the `feast materialize ..` command below, you will see a message <i>Materializing 2 feature views from 1995-01-01 01:01:01+00:00 to 2025-01-01 01:01:01+00:00 into the sqlite online store </i>  will be printed out.

Note that materialization step takes some time.. 

In [12]:
!feast materialize 1995-01-01T01:01:01 2025-01-01T01:01:01

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Materializing 2 feature views from 1995-01-01 01:01:01+00:00 to 2025-01-01 01:01:01+00:00 into the redis online store.

item_features:
100%|███████████████████████████████████████████████████████████| 670/670 [00:00<00:00, 4030.00it/s]
user_features:
100%|███████████████████████████████████████████████████████████| 649/649 [00:00<00:00, 4705.65it/s]


In [50]:
user_features

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories,user_id_raw,datetime,created
0,1,1,1,1,1,1,1,1,1,1,1,1,7,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
1,2,2,1,1,1,1,1,1,1,2,2,2,6,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
2,3,3,1,1,1,1,1,1,1,3,3,3,8,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
3,4,4,1,1,1,1,1,1,1,4,4,4,9,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
4,5,5,1,1,1,1,1,1,1,5,5,5,5,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,746,746,38,6,1,4,2,1,2,746,746,746,1457,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
645,747,747,46,6,2,4,2,2,3,747,747,747,1522,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
646,749,749,39,6,2,4,2,2,3,749,749,749,1562,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434
647,750,750,47,6,2,4,2,2,3,750,750,750,1581,2023-01-17 19:46:45.939170,2023-01-17 19:46:45.940434


In [28]:
import feast
feature_store = feast.FeatureStore(feature_repo_path)

In [54]:
feature_store.get_feature_view("user_features").features

[user_shops-Int32,
 user_profile-Int32,
 user_group-Int32,
 user_gender-Int32,
 user_age-Int32,
 user_consumption_2-Int32,
 user_is_occupied-Int32,
 user_geography-Int32,
 user_intentions-Int32,
 user_brands-Int32,
 user_categories-Int32,
 user_id-Int32]

In [45]:
res = feature_store.get_online_features(features=["user_features:user_id", "user_features:user_age"], entity_rows=[{"user_id_raw": 1}]).to_df()

In [49]:
res.to_numpy().reshape(-1, 3)

array([[ 1,  1, 32]])

### Explore Feature Repo Structure

In [13]:
import seedir as sd

sd.seedir(
    feature_repo_path,
    style="lines",
    itemlimit=10,
    depthlimit=3,
    exclude_folders=[".ipynb_checkpoints", "__pycache__"],
    sort=True,
)

feature_repo/
├─data/
│ ├─item_features.parquet
│ ├─registry.db
│ └─user_features.parquet
├─feature_store.yaml
├─item_features.py
└─user_features.py


## Redis ANN Index Setup

### Load Item Embeddings
We will load the pre-generated Item embeddings from file in preparation for loading into the Redis Server.

In [14]:
item_embeddings = Dataset(os.path.join(DATA_DIR, "item_embeddings.parquet")).to_ddf().compute()
item_embeddings.head()

,item_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,0.053040,-0.072800,0.029477,0.061942,-0.083255,-0.027927,0.040919,0.049997,-0.029252,...,0.049286,-0.087432,0.009700,-0.071424,0.024802,-0.012824,-0.045329,0.098826,0.028681,-0.000962
1,2,-0.022205,-0.073701,0.084334,-0.087279,0.050656,-0.088896,0.024600,0.115385,-0.007804,...,0.069289,0.089851,0.059622,0.002280,-0.102191,0.056542,0.067240,-0.019867,-0.049110,-0.099910
2,3,0.009324,0.004293,-0.021806,0.011273,0.020458,0.000370,0.035672,0.035399,-0.019392,...,-0.066249,-0.038092,0.028947,-0.004527,-0.007591,0.090114,-0.041121,0.030358,-0.016289,0.000246
3,4,0.028158,-0.015050,-0.010841,0.057088,-0.019317,0.015477,0.011777,-0.020076,-0.026838,...,-0.004810,-0.044477,-0.003194,0.012644,-0.043281,0.032553,-0.011370,0.024755,-0.060624,-0.010108
4,5,-0.041253,0.041424,0.004513,-0.033554,0.051555,-0.000238,0.004218,0.086332,-0.030636,...,-0.083374,0.008296,-0.003563,0.006302,0.002813,0.013661,0.040882,0.018407,-0.014042,-0.012307


In [4]:
import asyncio
import redis.asyncio as redis
from redis.commands.search.query import Query
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.field import VectorField

# Connect to the Redis client
host, port = os.environ["FEATURE_STORE_ADDRESS"].split(":")
redis_conn = redis.Redis(host=host, port=port)

In [6]:
# Define Redis ANN Index Params and Fields
INDEX_NAME = "candidate_index"
VECTOR_FIELD_NAME = "item_embedding"

vector_field = VectorField(
    VECTOR_FIELD_NAME,
    "HNSW", {
        "TYPE": "FLOAT32",
        "DIM": 64,
        "DISTANCE_METRIC": "IP",
        "INITIAL_CAP": len(item_embeddings),
    }
)

# Create ANN Index
await redis_conn.ft(INDEX_NAME).create_index(
    fields = [vector_field],
    definition= IndexDefinition(prefix=["ITEM:"], index_type=IndexType.HASH)
)

In [19]:
# Function to write item embeddings to Redis
async def write_item_embeddings(embs, n: int, redis_conn: redis.Redis):
    semaphore = asyncio.Semaphore(n)
    async def write(row):
        async with semaphore:
            item_id = int(row.pop("item_id"))
            entry = {
                "item_id": item_id,
                VECTOR_FIELD_NAME: np.array(row.values, dtype=np.float32).tobytes()
            }
            await redis_conn.hset(f"ITEM:{item_id}", mapping=entry)
    asyncio.gather(*[write(row[1]) for row in embs.iterrows()])

In [20]:
# Write embeddings to Redis ANN Index created above
await write_item_embeddings(item_embeddings, 100, redis_conn)

In [7]:
# Verify Index Construction
await redis_conn.ft(INDEX_NAME).info()

ResponseError: Unknown Index name

In [25]:
# Fetch an Item ID
item_ids = [key for key in await redis_conn.keys() if b"ITEM:" in key]
item_id = item_ids[0]

# Fetch a testing input vector
test_vector = await redis_conn.hget(item_id.decode("utf"), VECTOR_FIELD_NAME)

# Create a Redis VSS Query
query = Query(f"*=>[KNN 10 @{VECTOR_FIELD_NAME} $vec_param AS vector_score]")\
    .sort_by("vector_score")\
    .return_fields("id", "vector_score")\
    .dialect(2)

# Search for KNN
k_nearest_neighbors = await redis_conn.ft(INDEX_NAME).search(query, query_params={"vec_param": test_vector})

In [26]:
k_nearest_neighbors.docs

[Document {'id': 'ITEM:47', 'payload': None, 'vector_score': '0.944757223129'},
 Document {'id': 'ITEM:59', 'payload': None, 'vector_score': '0.950487792492'},
 Document {'id': 'ITEM:60', 'payload': None, 'vector_score': '0.94100278616'},
 Document {'id': 'ITEM:76', 'payload': None, 'vector_score': '0.936520278454'},
 Document {'id': 'ITEM:93', 'payload': None, 'vector_score': '0.951111078262'},
 Document {'id': 'ITEM:150', 'payload': None, 'vector_score': '0.933511078358'},
 Document {'id': 'ITEM:152', 'payload': None, 'vector_score': '0.943340182304'},
 Document {'id': 'ITEM:267', 'payload': None, 'vector_score': '0.946922540665'},
 Document {'id': 'ITEM:312', 'payload': None, 'vector_score': '0.949907541275'},
 Document {'id': 'ITEM:318', 'payload': None, 'vector_score': '0.950498878956'}]

### Next Steps
In this notebook we created our Feast Feature Store and setup the Redis ANN Index. Next, we will deploy our trained models into [Triton Inference Server (TIS)](https://github.com/triton-inference-server/server) with Merlin Systems library.

For the next step, move on to the [`02-Deploying-Online-Multi-Stage-Recsys-with-Triton.ipynb`](./02-Deploying-Online-Multi-Stage-Redsys-with-Triton.ipynb) notebook to deploy our saved models as an ensemble to TIS and obtain prediction results for a given request.